In [59]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from matplotlib.backends.backend_pdf import PdfPages
from astropy.wcs import WCS
from astropy import units as u
from astropy.coordinates import SkyCoord
#pd.options.mode.copy_on_write = True
from astropy.table import Table
from astropy.cosmology import FlatLambdaCDM
import multiprocessing as mp
from tqdm import tqdm
from astropy.time import Time
cosmo = FlatLambdaCDM(name='Planck18', H0=67.66, Om0=0.30966, Tcmb0=2.7255, Neff=3.046, m_nu=[0.  , 0.  , 0.06]* u.eV, Ob0=0.04897)
cores = 8

In [9]:
import base64
import gzip
import io
import logging
import os
import glob
import time
from copy import deepcopy
from datetime import datetime
from typing import Mapping, Optional

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from astropy.time import Time
from astropy.visualization import (
    AsymmetricPercentileInterval,
    ImageNormalize,
    LinearStretch,
    LogStretch,
)
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from colorama import init as colorama_init
from colorama import Fore, Style

#import threading
#from threading import Thread
import multiprocessing as mp

In [10]:
import GetLightcurve
import SendToSkyportal

In [51]:
crossmatch_df = pd.read_csv('../CrossMatch/DESIRT_one_arcsecond_crossmatch_final_list.csv')
DESIRT_lc = pd.read_csv('../LightCurves/DESIRT_lightcurves_final_list.csv')

In [12]:
base_name = "DESIRT_PIPE"
group_ids = [1721]
filter_ids = [1157]
instrument_id = 54
stream_id = 1006
token_skyportal = os.getenv("FRITZ_TOKEN")
endpoint = "fritz.science"
origin = "DESIRT_BOT"
filter_dict = {b'g': "desg", b'i': "desi", b'r': "desr", b'z': "desz", b'u': "desu"}

In [13]:
def api(method, path, data):
    headers = {'Authorization': f'token {token_skyportal}'}
    response = requests.request(method, path, params=data, headers=headers)
    return response

In [79]:
def api_post(method, path, data):
    headers = {'Authorization': f'token {token_skyportal}'}
    response = requests.request(method, path, json=data, headers=headers)
    return response

In [38]:
def get_favorites(pageNumber):
    """
    Grab all favorites from DESIRT
    """
    data = {
        "numPerPage" : 500,
        "group_ids": group_ids,
        "listName": "favorites",
        "pageNumber": pageNumber
    }
    response = api("GET", f"https://{endpoint}/api/sources", data)
    return response

In [89]:
def request_f_phot(ObjID, date):
    """
    Request ZTF forced phot for a given source.
    """
    
    payload = {
        "start_date" : date[0],
        "end_date" : date[1],
        "position" : "Source (object's coordinates)",
        "request_type" : "forced_photometry",
    }
    
    data = {
        "obj_id" : ObjID,
        "allocation_id" : 1052,
        "target_group_ids" : group_ids,
        "payload" : payload,
    }
    response = api_post("POST", f"https://{endpoint}/api/followup_request", data)
    return response

In [110]:
def get_start_end_dates(ObjID, crossmatch):
    df_photometry = GetLightcurve.full_dataframe([ObjID, crossmatch])[['STANDARD_NAME', 'SEMESTER_NAME', 'SCIENCE_NAME', 'TEMPLATE_NAME', 'MJD_OBS', 'FILTER', 'DETPOS',
       'X_OBJ', 'Y_OBJ', 'X_FPHOT', 'Y_FPHOT', 'X_nSTAR', 'Y_nSTAR', 'X_nGAL',
       'Y_nGAL', 'X_ALT', 'Y_ALT', 'MAG_ALT', 'MAGERR_ALT', 'SNR_ALT',
       'CNNSCORE_ALT', 'ZMIN_TEMP', 'ZMAX_TEMP', 'ZMIN_SCI', 'ZMAX_SCI',
       'ZMIN_DIFF', 'ZMAX_DIFF', 'APER_FPHOT', 'ZP_FPHOT', 'LIM_MAG3',
       'LIM_MAG5', 'MAG_FPHOT', 'MAGERR_FPHOT', 'SNR_FPHOT', 'STATUS_FPHOT',
       'REAL', 'LIMIT']]
    
    df_photometry.drop_duplicates(subset = ["MAG_FPHOT", "MAGERR_FPHOT", "MJD_OBS", "FILTER"], inplace = True)
    filters = [filter_dict[i] for i in df_photometry["FILTER"]]
    
    obs_dates = df_photometry["MJD_OBS"]
    buffer = 28
    obs_dates_pre = np.min(obs_dates) + buffer
    obs_dates_post = np.max(obs_dates) - buffer

    obs_dates = Time([obs_dates_pre, obs_dates_post], format = "mjd")
    obs_dates_datetime = obs_dates.to_datetime()
    
    obs_dates_strings = []
    for i in obs_dates_datetime:
        temp = i.isoformat().split("T")
        temp_hours = temp[1].split(".")[0]
        temp_date = temp[0]
        obs_dates_strings.append("{} {}".format(temp_date, temp_hours))
        
    return obs_dates_strings

In [40]:
response = get_favorites(1)
response2 = get_favorites(2)

In [43]:
sources_list = response.json()["data"]["sources"]
sources_list.extend(response2.json()["data"]["sources"])

In [44]:
len(sources_list)

649

In [45]:
ObjIDs = []
for i in sources_list:
    ObjIDs.append(i["id"])
ObjIDs = np.array(ObjIDs)
len(ObjIDs)

649

In [46]:
np.save("interesting", ObjIDs)

In [52]:
ObjID = ObjIDs[0]
crossmatch = crossmatch_df

In [53]:
ObjID

'A202204211423123m005116'

In [54]:
df_photometry = GetLightcurve.full_dataframe([ObjID, crossmatch])[['STANDARD_NAME', 'SEMESTER_NAME', 'SCIENCE_NAME', 'TEMPLATE_NAME', 'MJD_OBS', 'FILTER', 'DETPOS',
       'X_OBJ', 'Y_OBJ', 'X_FPHOT', 'Y_FPHOT', 'X_nSTAR', 'Y_nSTAR', 'X_nGAL',
       'Y_nGAL', 'X_ALT', 'Y_ALT', 'MAG_ALT', 'MAGERR_ALT', 'SNR_ALT',
       'CNNSCORE_ALT', 'ZMIN_TEMP', 'ZMAX_TEMP', 'ZMIN_SCI', 'ZMAX_SCI',
       'ZMIN_DIFF', 'ZMAX_DIFF', 'APER_FPHOT', 'ZP_FPHOT', 'LIM_MAG3',
       'LIM_MAG5', 'MAG_FPHOT', 'MAGERR_FPHOT', 'SNR_FPHOT', 'STATUS_FPHOT',
       'REAL', 'LIMIT']]
    
df_photometry.drop_duplicates(subset = ["MAG_FPHOT", "MAGERR_FPHOT", "MJD_OBS", "FILTER"], inplace = True)
filters = [filter_dict[i] for i in df_photometry["FILTER"]]

In [63]:
obs_dates = df_photometry["MJD_OBS"]
buffer = 28
obs_dates_pre = np.min(obs_dates) + buffer
obs_dates_post = np.max(obs_dates) - buffer

obs_dates = Time([obs_dates_pre, obs_dates_post], format = "mjd")
obs_dates_datetime = obs_dates.to_datetime()

In [75]:
obs_dates_strings = []
for i in obs_dates_datetime:
    temp = i.isoformat().split("T")
    temp_hours = temp[1].split(".")[0]
    temp_date = temp[0]
    obs_dates_strings.append("{} {}".format(temp_date, temp_hours))

In [90]:
response = request_f_phot(ObjID, obs_dates_strings)

In [92]:
response

<Response [200]>

In [107]:
queue = [[ObjIDs[i], crossmatch_df] for i in range(len(ObjIDs))]
len(queue)

649

In [113]:
response_list = []
for i in tqdm(queue):
    start_end_dates = get_start_end_dates(i[0], i[1])
    response_list.append(request_f_phot(i[0], start_end_dates))
    time.sleep(0.1)

100%|██████████| 649/649 [15:04<00:00,  1.39s/it]


In [102]:
len(ObjIDs)

649